In [ ]:
import sys
import random

def decoupe_tab(li) :
    l = li.split()
    return l

def import_distrib_ressources(path):
    tab = [] #tableau données
    f = open(path)
    ligne = f.readline()
    while ligne.strip() != "" : #jusqu'a fin du fichier
        tab = tab + (decoupe_tab(ligne)) #ajout ligne dans tableau ['nom','nombre']
        ligne = f.readline() #change de ligne
    f.close()
    i = 0
    tabRessources =[] #tableau ressources (seulement ressources)
    while (i < len(tab)):
        nomRessource = tab[i] #stock nom ress
        i += 1 #acces au nb de ressource
        nbRessource = tab[i] #stock nb ress
        j = 0
        while(j < int(nbRessource)):#ajout de la ress en fonction de sa quantité, cast car string
            tabRessources.append(nomRessource) #ajout dans le tabRessources le nom de la ressources autant de fois que son nb
            j += 1
        i+=1
    return tabRessources

def init_coffre(path):
    tab = [] #tableau vide
    coffre = {} #dico vide
    f = open(path)
    ligne = f.readline()
    while ligne.strip() != "" : #remplir le tableau avec les donnees du fichier texte
        tab = tab + (decoupe_tab(ligne)) #ajout ligne dans tab ['nom','nombre']
        ligne = f.readline() #change de ligne
    f.close()
    
    i = 0
    while (i < len(tab)):
        nomRess = tab[i]
        coffre[nomRess]=0
        i+=2
    return coffre

def fixe_longueur(chaine, long):
    diff = 0
    if len(chaine) < long :
        diff = long - len(chaine)
        chaine = diff*' ' + chaine
    return chaine

def liste_items(coffre):
    return list(coffre.keys())

def ouvre_coffre(coffre):
    l = liste_items(coffre)
    for i in range(0,len(coffre)):
        tab = fixe_longueur(l[i], 15) + '     |     ' + str(coffre[l[i]])
        print(tab)

def ajoute(coffre, item):
    coffre[item]+=1

def ajoute_plusieurs(coffre, panier) :
    for i in range(0, len(panier)):
        ajoute(coffre, panier[i])

def est_present(coffre, item):
    return coffre[item] >= 1

def sont_presents(coffre, liste_item):
    j = 0
    for i in range(0, len(liste_item)):
        if est_present(coffre, liste_item[i]) == True:
            j+=1
    if j == len(liste_item):
        return True
    else:
        return False

def retire(coffre, item):
    coffre[item]-=1

def retire_plusieurs( coffre, liste_item):
    for i in range(0, len(liste_item)):
        if sont_presents(coffre, liste_item):
            retire(coffre, liste_item[i])
    return True

def glaner(ressources):
    tabGlan = []
    for i in range(0,5):
        i = random.randint(0, len(ressources))
        tabGlan.append(ressources[i])
    return tabGlan

def import_regles_craft(path):
    tab = [] #tableau items fabric
    règles_craft = {} # dico règles
    i = 0
    f = open(path)
    ligne = f.readline()
    while ligne.strip() != "" : #jusqu'a fin du fichier
        règles_craft[decoupe_tab(ligne)[0]] = 0 #met les objets a fabriquer en clés
        for i in range(1, len(decoupe_tab(ligne))):
            tab.append(decoupe_tab(ligne)[i]) #rajoute au tableau items les items sur chq ligne
            règles_craft[decoupe_tab(ligne)[0]] = tab #met le tableau en valeur dans le dico
        tab = [] # vide le tableau
        ligne = f.readline() #change de ligne
    f.close()
    return règles_craft

def craft_possible(coffre, regles_craft, item) :
    if sont_presents(coffre, regles_craft[item]):
        return True
    else :
        return False

def craft(coffre, regles_craft, item):
    if craft_possible(coffre, regles_craft, item):
        for i in range(0, len(regles_craft[item])):
            coffre[regles_craft[item][i]] -=1
        coffre[item] +=1
        return True
    else :
        return False

def manger(coffre, item, PdV):
    if est_present(coffre, item):
        if item == 'pain':
            PdV = PdV + 10
            retire(coffre, item)
            return PdV
        elif item == 'tomate':
            PdV = PdV + 5
            retire(coffre, item)
            return PdV
        elif item == 'ble':
            PdV = PdV + 1
            retire(coffre, item)
            return PdV
        else:
            print("L'item "+ item +" n'est pas comestible !")
            return PdV
    else:
        print("L'item "+ item+ " n'est pas dans votre coffre !")
        return PdV

def maj_PdV(coffre, pdv) :
    if est_present(coffre, 'vetement') and est_present(coffre, 'lit') == False and est_present(coffre, 'hutte') == False:
        pdv -= 45
        return pdv
    elif est_present(coffre, 'lit') and est_present(coffre, 'vetement') == False and est_present(coffre, 'hutte') == False:
        pdv -= 43
        return pdv
    elif est_present(coffre, 'hutte') and est_present(coffre, 'lit') == False and est_present(coffre, 'vetement') == False:
        pdv -= 35
        return pdv
    elif sont_presents(coffre, ['vetement', 'lit']) and est_present(coffre, 'hutte') == False:
        pdv -= 38
        return pdv
    elif sont_presents(coffre, ['vetement', 'hutte']) and est_present(coffre, 'lit') == False:
        pdv -= 30
        return pdv
    elif sont_presents(coffre, ['hutte', 'lit']) and est_present(coffre, 'vetement') == False:
        pdv -= 28
        return pdv
    elif sont_presents(coffre, ['vetement', 'lit', 'hutte']):
        pdv -= 23
        return pdv
    else :
        pdv -= 50
        return pdv

def est_dans(chaine, liste_de_chaine ):
    if chaine in liste_de_chaine:
        return True
    else:
        return False

def saisie_controlee(message, admissible) :
    print(message)
    m = input()
    while est_dans(m, admissible) == False:
        print('Saisie non reconnue')
        print(message)
        m = input()
    return m

def choix_item(coffre) :
    return saisie_controlee("Choisissez un item:", liste_items(coffre))

def partie():
    coffre = init_coffre("data/ressources")
    regles_craft = import_regles_craft("data/regles_craft")
    ressources = import_distrib_ressources("data/ressources")
    PDV = 1000
    jouer = 'oui'
    
    while jouer == 'oui' and PDV > 0:
        gla = glaner(ressources, )
        ajoute_plusieurs(coffre, gla)

        print("Craft & Survive")
        print("----------------")
        print("Voici votre coffre :")
        ouvre_coffre(coffre)
        print("----------------")
        print("Voici vos points de vie (PDV) : " + str(PDV))

        ch_craft = saisie_controlee("Souhaitez-vous crafter ?", ["oui","non"])
        if ch_craft == 'oui': #si on veut crafter
            print('La liste des item est :')
            print(liste_items(coffre))
            print('Que souhaitez -vous crafter ?')
            choix_item_craft = choix_item(coffre)
            if craft(coffre, regles_craft, choix_item_craft): #si le craft est possible
                print("Craft effectué.")
            else:
                print('Craft impossible.')

        ch_manger = saisie_controlee("Souhaitez-vous manger ?", ["oui","non"])
        if ch_manger == 'oui':
            ch_nourriture = saisie_controlee("Que souhaitez-vous manger ? (ble, pain, tomate)", ["ble","pain","tomate"])
            PDV = manger(coffre, ch_nourriture, PDV)

        PDV = maj_PdV(coffre, PDV)
        jouer = saisie_controlee("Souhaitez-vous continuer ?", ["oui","non"])
        if jouer == 'non':
            break
    
    print('Vous avez '+str(PDV)+' points de vie.')

partie()

Craft & Survive
----------------
Voici votre coffre :
       aiguille     |     0
          beche     |     0
            ble     |     0
           bois     |     0
           clou     |     0
          corde     |     0
          coton     |     0
            fer     |     0
     graine_ble     |     0
   graine_coton     |     0
  graine_tomate     |     0
          hutte     |     0
            lit     |     0
        marteau     |     0
metier_a_tisser     |     0
          osier     |     1
         paille     |     4
           pain     |     0
         panier     |     0
          tissu     |     0
         tomate     |     0
       vetement     |     0
----------------
Voici vos points de vie (PDV) : 1000
Souhaitez-vous crafter ?
